# /tl/ Clusters

This notebook includes functions that allow us to obtain voicing, rise time, and relative intensity from /tl/ clusters. These functions make use of the [Parselmouth](https://buildmedia.readthedocs.org/media/pdf/parselmouth/latest/parselmouth.pdf) and [audiolabel](https://github.com/rsprouse/audiolabel) packages.

### Libraries

In [ ]:
import os  # For some basic file functionality (creating/removing/parsing file paths)
import re  # For regular expessions
import numpy as np  # For advanced numerical calculations, multi-dimensional arrays

#import audiolabel
from audiolabel import read_label  # For reading TextGrid files
from phonlab.utils import dir2df  # For finding directories and pulling data
import parselmouth  # For incorporating Praat features in this notebook

# To get general Praat functionality where native features don't exist
from parselmouth.praat import call as pcall

import pandas as pd  # For creating managing dataframes

### Functions

In [ ]:
def get_voicing(row, psnd):
    '''
    Calculates the voice percentage of a portion of an audio file. It's designed to be called from
    pd.DataFrame.apply. 
    
    Note that Praat typically takes some time from the beginning of the audio to identify the pitch, 
    which is needed to get the voicing; thus, we will add some buffer time to our sound called 'pad'.
    
    Parameters
    ----------
    row: DataFrame row
        (Identifies a portion of audio within the time period specified. Each row needs to have a 't1_ph'
        and a 'next_ph_t2' attributes, to identify the start and end point of the cluster under evaluation.
        A 'sex' ('male' or 'female') attribute is require for each row to set the pitch floor.)
    
    psnd: Parselmouth Sound
        (This is the portion of the audio to be analyzed from 't1_ph' to 'next_ph_t2'.)
    
    Returns
    -------
    voiced: float
        (Percentage of voicing between 't1_ph' and 'next_ph_t2'.)
    '''
    
    pad = 0.5  # Add buffer time needed to identify the pitch
    s = psnd.extract_part(row.t1_ph-pad, row.next_ph_t2+pad)  # Select the audio portion of interest
    pitch_floor = 100  # Select the default pitch floor (for female speakers)
    if row.sex == 'male':  # Change the pitch floor for male speakers
        pitch_floor = 70
    
    # From sound 's', obtain pitch object
    pitch = pcall(
        s,
        'To Pitch (cc)...',
        0.001,  # Time step (s) [default = 0.25 sec]
        pitch_floor, # Pitch floor (Hz) [default = 75 Hz]
        15,  # Max. number of candidates [default]
        0,  # Very accurate (unselected) [default = unselected, i.e. off]
        0.03,  # Silence threshold [default]
        0.45,  # Voicing threashold [default]
        0.01,  # Octave cost [default]
        0.35,  # Octave-jump cost [default]
        0.14,  # Voiced/unvoided cost [default]
        250.0,  # Pitch ceiling (Hz) [default = 600 Hz]
    )
    
    # From sound 's' and 'pitch' object, obtain PointProcess object (i.e. 'pulses')
    pulses = pcall(
        [s, pitch], 
        'To PointProcess (cc)'
    )
    
    # From sound 's', 'pitch' object, and PointProcess object (i.e. 'pulses'), obtain Voice report (i.e. 'voicing')
    voicing = pcall(
        [s, pitch, pulses],
        'Voice report',
        pad,  # Time range (s) (start; add padding)
        (row.next_ph_t2-row.t1_ph)+pad,  # Time range (s) (end; add padding)
        pitch_floor,  # Pitch floor (Hz) (start)
        600.0,  # Pitch floor (Hz) (end)
        1.3,  # Maximum period factor [default]
        1.6,  # Maximum amplitude factor [default]
        0.03,  # Silence threshold [default]
        0.45  # Voicing threshold [default]
    )
    
    # From Voice report (i.e. 'voicing'), obtain voicing percentage
    m = re.search('unvoiced frames: (?P<percent>\d+\.\d*|\d+)', voicing)
    if m:
        voiced = 100.0-float(m.group('percent'))
    else:
        voiced = np.nan
    
    return voiced

In [ ]:
def get_relative_intensity(row, psnd):
    '''
    Calculates the relative intensity (difference) of two segments.
    
    Parameters
    ----------
    row: DataFrame row
        (Identifies a portion of audio within the time periods specified. Each row needs to have a 't1_ph',
        't2_ph', and a 'next_ph_t2' attributes, to identify the start and end point of the segments in the
        cluster.)
    
    psnd: Parselmouth Sound
       (This is the portion of the audio to be analyzed from 't1_ph' to 'next_ph_t2'.)
    
    Returns
    -------
    rel_intensity: float
       (The relative intensity or intensity difference).
    '''

    t_s = psnd.extract_part(row.t1_ph, row.t2_ph)  # Obtain /t/ segment
    l_s = psnd.extract_part(row.t2_ph, row.next_ph_t2)  # Obtain /l/ segment
    
    return t_s.get_intensity()-l_s.get_intensity()

In [ ]:
def get_rise_time(row, psnd):
    '''
    Calculate the rise time in a fricative segment. Rise time is the point in time when a fricative reaches
    its highest amplitude. Because rise time is a typically obtained from fricative sounds, we will obtain
    this measurement from the lateral segment of the /tl/ cluster, not the stop portion.
    
    Parameters
    ----------
    row: DataFrame row
        (Identifies a portion of audio within the time periods specified. Each row needs to have a 't2_ph' 
        and a 'next_ph_t2' attributes, to identify the start and end point of the segments in the cluster.)
    
    psnd: Parselmouth Sound
       (This is the portion of the audio to be analyzed from 't2_ph' to 'next_ph_t2'.)
    
    Returns
    -------
    rise_time: float
    '''
    
    s = psnd.extract_part(row.t2_ph, row.next_ph_t2)
    rise_time = (pcall(
        s, 
        'Get time of maximum...', 
        0.0,  # Time range (s) (start) [default] (selects entire sound)
        0.0,  # Time range (s) (end) [default] (selects entire sound)
        "Parabolic"  # Interpolation
    )*1000)  # Multiply times 1000 to convert the value (in seconds) to miliseconds
    
    return rise_time

### Start creating dataframe

In [ ]:
# For more information on using audiolabel and managing directories, refer to Ronald Sprouse's documentation
# on audiolabel (https://github.com/rsprouse/audiolabel)

# Identify location of data
datadir = './Data'
dirpat = '(?P<subject>S\d+)' # Give each file a subject number, according to file name.

# Identify files of interest
fdf = dir2df(datadir, dirpat=dirpat, fnpat='\.wav$', addcols=['barename','dirname'])
speaker_sex = './speaker_sex.csv'
speaker_sex = pd.read_csv(speaker_sex, encoding = 'utf-8')
fdf = fdf.merge(speaker_sex, on='subject', how='left')

# Identify data (wav and textgrid files)
wav_suffix = '_words.wav'
tg_suffix = '_words_Spanish_aligned.TextGrid'

In [ ]:
def func_executor(participant):
    '''
    This function executor applies the functions defined earlier to the data identify in the previous cell.
    '''
    
    print(participant)  # Print participant number to follow progress and isolate potential errors
    
    wav_file = os.path.join(datadir,participant.relpath,participant.relpath+wav_suffix)
    tg_file = os.path.join(datadir,participant.relpath,participant.relpath+tg_suffix)

    psnd = parselmouth.Sound(wav_file) 

    [phdf,wddf] = read_label(tg_file, 'praat')

    word_info = './word_info.csv'  #Identify file's location
    widf = pd.read_csv(word_info, encoding = 'utf-8')  #Create df for said file using UTF-8
    widf.Token = widf.Token.str.upper()  #Make words uppercase to match wdpf

    merged_wddf = wddf.merge(widf, left_on='label', right_on='Token', how='left').sort_values(by='t1')

    phdf = phdf.assign(  # Create new column
        next_ph=phdf.label.shift(periods=-1,axis=0,fill_value=''),  #Roll phone's label to create cluster
        next_ph_t2=phdf.t2.shift(periods=-1,axis=0,fill_value=np.nan))  # Roll phone's t2 

    phdf_tl = phdf[(phdf.label=='t')&(phdf.next_ph=='l')]

    phwddf = pd.merge_asof(
        phdf_tl.rename(
            columns={'t1':'t1_ph'}), 
        merged_wddf.rename(
            columns={'t1':'t1_wd'}), 
        left_on='t1_ph', 
        right_on='t1_wd', 
        suffixes=['_ph', '_wd']
    )

    phwddf = phwddf.assign(
        sex=participant.sex
    )

    phwddf = phwddf.assign(
        voicing=phwddf.apply(
            get_voicing,
            args=([psnd]), 
            axis=1
        ),
        rel_intensity=phwddf.apply(
            get_relative_intensity,
            args=([psnd]),
            axis=1
        ),
        rise_time=phwddf.apply(
            get_rise_time, 
            args=([psnd]), 
            axis=1
        )
    )
        
    return phwddf

In [ ]:
# Obtain all measurements

measurements_df = pd.concat(fdf.apply(func_executor, axis=1).tolist())

In [ ]:
# Identify any empty cells or cells with NaN -- an empty df means there were no issues

measurements_df[np.any(measurements_df.isna(), axis=1)]

In [ ]:
# Safe measurement results as 'tl.csv' under a folder called 'Results'

export_csv = new_df.to_csv('Results/tl.csv', index=None, header=True)